In [1]:
# Consolidated imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from libpysal.weights import Queen, lag_spatial

from esda.moran import Moran

import statsmodels.api as sm
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'

# Spatial Feature Engineering Pipeline
This notebook demonstrates spatial feature engineering for San Francisco block groups. It loads cleaned data, computes a Queen weights graph, and evaluates four spatial feature types (coordinates, spatial lag, Moran eigenvectors, node2vec embeddings) in OLS regression, reporting fit and Moran's I of residuals for each.

## 1. Load Cleaned Dataset
Load tabular and spatial data, merge into a GeoDataFrame, and clean missing/infinite values.

In [2]:
# Load tabular and spatial data
data = pd.read_csv('data/incidents/InputDataset.csv', dtype={'BGFIPS10': object}).set_index('BGFIPS10')
geo = gpd.read_parquet('data/incidents/SF_BlockGroups10_Cleaned.parquet').set_index('BGFIPS10')
gdf = gpd.GeoDataFrame(data, geometry=geo.geometry)
gdf = gdf.replace([np.inf, -np.inf], np.nan).dropna()

KeyError: "None of ['BGFIPS10'] are in the columns"

## 2. Compute Queen Weights Graph
Build a Queen contiguity weights object (row-standardized) for use throughout the notebook.

In [ ]:
# Compute Queen weights (row-standardized)
queen = Queen.from_dataframe(gdf, ids=gdf.index, silence_warnings=True)
queen.transform = "R"

## 3. Baseline OLS and Moran's I
Fit a simple OLS model (no spatial features) and compute Moran's I of residuals.

In [ ]:
# Baseline OLS (no spatial features)
YVar = 'DrunkP1k'
XVars = ['pHHPov','BarPSqMi','RetailPSqMi','pMale','VacantHU']
y = gdf[YVar]
X = gdf[XVars]
X = sm.add_constant(X)
fit = sm.OLS(y, X).fit()
print(fit.summary())

In [ ]:
# Compute residuals and Moran's I
residuals = y - fit.fittedvalues
moran_baseline = Moran(residuals.values, queen)
print('Baseline residual Moran I:', moran_baseline.I, moran_baseline.p_sim)

In [ ]:
from splot.esda import plot_moran
plot_moran(moran_baseline, zstandard=True)
plt.title('Moran Plot of Residuals (Baseline OLS)')
plt.show()

## 4. Spatial Feature Engineering
We now compute and add each spatial feature class, then run OLS and Moran's I diagnostics for each.

### 4a. Coordinates
Add centroid coordinates as features and rerun OLS.

In [ ]:
gdf['centroid_x'] = gdf.geometry.centroid.x
gdf['centroid_y'] = gdf.geometry.centroid.y
X_coord = sm.add_constant(gdf[XVars + ['centroid_x', 'centroid_y']])
fit_coord = sm.OLS(y, X_coord).fit()
print(fit_coord.summary())

In [ ]:
# Compute residuals and Moran's I for coordinates model
res_coord = y - fit_coord.fittedvalues
moran_coord = Moran(res_coord.values, queen)
print('Residual Moran I (coords):', moran_coord.I, moran_coord.p_sim)

In [ ]:
# Plot Moran's I of residuals for coordinates model
from splot.esda import plot_moran
plot_moran(Moran(res_coord.values, queen), zstandard=True)
plt.title('Moran Plot of Residuals (Coordinates)')
plt.show()

### 4b. Spatial Lag of Explanatory Variables
Add spatial lag features for each explanatory variable and rerun OLS.

In [ ]:
def spatial_lag(df, columns, w):
    lagged = {}
    for col in columns:
        lagged[f"{col}_lag"] = lag_spatial(w, df[col].values)
    return pd.DataFrame(lagged, index=df.index)

# Compute spatial lags and add to gdf
gdf_lags = spatial_lag(gdf, XVars, queen)
for col in gdf_lags.columns:
    gdf[col] = gdf_lags[col]
X_lag = sm.add_constant(gdf[XVars + list(gdf_lags.columns)])
fit_lag = sm.OLS(y, X_lag).fit()
print(fit_lag.summary())

In [ ]:
# Compute residuals and Moran's I for spatial lag model
res_lag = y - fit_lag.fittedvalues
moran_lag = Moran(res_lag.values, queen)
print('Residual Moran I (spatial lag):', moran_lag.I, moran_lag.p_sim)

In [ ]:
# Plot Moran's I of residuals for spatial lag model
plot_moran(Moran(res_lag.values, queen), zstandard=True)
plt.title('Moran Plot of Residuals (Spatial Lag)')
plt.show()

### 4c. Moran Eigenvector Spatial Filtering (MESF)
Compute Moran eigenvectors and add top k as features, then rerun OLS.

In [ ]:
def compute_mesf(gdf, w, k=10):
    idx = gdf.index.to_list()
    n = len(idx)
    A = w.full()[0]
    I = np.eye(n)
    one = np.ones((n, 1))
    M = I - (one @ one.T) / n
    S = 0.5 * (M @ A @ M + M @ A.T @ M)
    evals, evecs = np.linalg.eigh(S)
    order = np.argsort(evals)[::-1]
    evals, evecs = evals[order], evecs[:, order]
    mem_cols = [f'MEM_{i}' for i in range(k)]
    mem_df = pd.DataFrame(evecs[:, :k], index=gdf.index, columns=mem_cols)
    return mem_df

# Compute MESF and add to gdf
mem_df = compute_mesf(gdf, queen, k=50)
for col in mem_df.columns:
    gdf[col] = mem_df[col]
X_mesf = sm.add_constant(gdf[XVars + list(mem_df.columns)])
fit_mesf = sm.OLS(y, X_mesf).fit()
print(fit_mesf.summary())

In [ ]:
# Compute residuals and Moran's I for MESF model
res_mesf = y - fit_mesf.fittedvalues
moran_mesf = Moran(res_mesf.values, queen)
print('Residual Moran I (MESF):', moran_mesf.I, moran_mesf.p_sim)

In [ ]:
# Plot Moran's I of residuals for MESF model
plot_moran(Moran(res_mesf.values, queen), zstandard=True)
plt.title('Moran Plot of Residuals (MESF)')
plt.show()

### 4d. Graph Embeddings with PyTorch Geometric GCN
In this section, we use a GCN autoencoder to generate spatial graph embeddings for each block group. We first define the model, then perform a grid search to select the optimal embedding size, and finally run the optimized model to generate features for OLS regression.

#### Step 1: Define the GCN Autoencoder Model
We first define the GCN autoencoder architecture, which will be used to learn node embeddings from the Queen contiguity graph and explanatory variables.

In [ ]:
# Prepare node features: explanatory variables only
scaler = StandardScaler()
X_feats = scaler.fit_transform(gdf[XVars])
x = torch.tensor(X_feats, dtype=torch.float32)

# Graph structure from Queen weights
G_nx = queen.to_networkx()
edge_index = torch.tensor(list(G_nx.edges)).t().contiguous()

# Define GCN autoencoder model
class GCN_AE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.encoder = GCNConv(in_channels, hidden_channels)
        self.decoder = torch.nn.Linear(hidden_channels, in_channels)
    def forward(self, x, edge_index):
        z = F.relu(self.encoder(x, edge_index))
        x_hat = self.decoder(z)
        return x_hat, z

#### Step 2: Grid Search for Optimal Embedding Size
We perform a grid search over several candidate embedding sizes (e.g., 4, 8, 16, 32, 48, 64, 128). For each size, we train the autoencoder, use the embeddings as features in OLS, and record model fit (AIC) and residual spatial autocorrelation (Moran's I and p-value).

In [ ]:
# Grid search for best embedding size
dims = [4, 8, 16, 32, 48, 64, 128]
results = []

for dim in dims:
    # Train GCN autoencoder with 'dim' hidden units
    model = GCN_AE(x.shape[1], dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        x_hat, z = model(x, edge_index)
        loss = F.mse_loss(x_hat, x)
        loss.backward()
        optimizer.step()
    embeddings = z.detach().numpy()
    for i in range(embeddings.shape[1]):
        gdf[f'gcn_ae_embed_{i}'] = embeddings[:, i]
    X_gcn_ae = sm.add_constant(gdf[XVars + [f'gcn_ae_embed_{i}' for i in range(embeddings.shape[1])]])
    fit_gcn_ae = sm.OLS(y, X_gcn_ae).fit()
    print(f'Embedding dim {dim} OLS summary:')
    print(fit_gcn_ae.summary())

In [ ]:
res_gcn_ae = y - fit_gcn_ae.fittedvalues
moran_I = Moran(res_gcn_ae.values, queen)
aic = fit_gcn_ae.aic
print(f'Embedding dim {dim}: AIC={aic:.2f}, Moran I={moran_I.I:.4f}, p-value={moran_I.p_sim:.4f}')

In [ ]:
plot_moran(Moran(res_gcn_ae.values, queen), zstandard=True)
plt.title(f'Moran Plot of Residuals (GCN AE dim {dim})')
plt.show()

#### Step 3: Run Final Optimized Model
After selecting the best embedding size (lowest residual Moran's I and competitive AIC), we retrain the GCN autoencoder with this size and use the resulting embeddings as features in the final OLS regression.

In [ ]:
# Final model: set best_dim to the optimal embedding size found above (e.g., best_dim = 48)
best_dim = 48  # Replace with your selected value from grid search
model = GCN_AE(x.shape[1], best_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    x_hat, z = model(x, edge_index)
    loss = F.mse_loss(x_hat, x)
    loss.backward()
    optimizer.step()
embeddings = z.detach().numpy()
for i in range(embeddings.shape[1]):
    gdf[f'gcn_ae_embed_{i}'] = embeddings[:, i]
X_gcn_ae = sm.add_constant(gdf[XVars + [f'gcn_ae_embed_{i}' for i in range(embeddings.shape[1])]])
fit_gcn_ae = sm.OLS(y, X_gcn_ae).fit()
print(fit_gcn_ae.summary())

In [ ]:
# Compute residuals and Moran's I for final GCN AE model
res_gcn_ae = y - fit_gcn_ae.fittedvalues
moran = Moran(res_gcn_ae.values, queen)
print(f'Residual Moran I (final GCN AE): {moran.I:.4f}, p-value: {moran.p_sim:.4f}')

In [ ]:
# Plot Moran's I of residuals for final GCN AE model
plot_moran(Moran(res_gcn_ae.values, queen), zstandard=True)
plt.title('Moran Plot of Residuals (Final GCN AE)')
plt.show()

# Model Comparison and Summary
In this section, we compare all fitted models (including baseline, spatial features, and VIF-selected models) using AIC and Moran's I of residuals.

In [ ]:
# Summary of fit metrics and Moran's I for all models (R_squared, R_squared_adjusted, and AIC)
summary_rows = []

try:
    summary_rows.append({
        'Model': 'Baseline OLS',
        'R_squared': fit.rsquared,
        'R_squared_adj': fit.rsquared_adj,
        'AIC': fit.aic,
        'Moran I': moran_baseline.I,
        'p-value': moran_baseline.p_sim
    })
except Exception:
    pass

try:
    summary_rows.append({
        'Model': 'Coordinates',
        'R_squared': fit_coord.rsquared,
        'R_squared_adj': fit_coord.rsquared_adj,
        'AIC': fit_coord.aic,
        'Moran I': moran_coord.I,
        'p-value': moran_coord.p_sim
    })
except Exception:
    pass

try:
    summary_rows.append({
        'Model': 'Spatial Lag',
        'R_squared': fit_lag.rsquared,
        'R_squared_adj': fit_lag.rsquared_adj,
        'AIC': fit_lag.aic,
        'Moran I': moran_lag.I,
        'p-value': moran_lag.p_sim
    })
except Exception:
    pass

try:
    summary_rows.append({
        'Model': 'MESF',
        'R_squared': fit_mesf.rsquared,
        'R_squared_adj': fit_mesf.rsquared_adj,
        'AIC': fit_mesf.aic,
        'Moran I': moran_mesf.I,
        'p-value': moran_mesf.p_sim
    })
except Exception:
    pass

try:
    summary_rows.append({
        'Model': 'GCN AE',
        'R_squared': fit_gcn_ae.rsquared,
        'R_squared_adj': fit_gcn_ae.rsquared_adj,
        'AIC': fit_gcn_ae.aic,
        'Moran I': moran.I,
        'p-value': moran.p_sim
    })
except Exception:
    pass

summary = pd.DataFrame(summary_rows)
print(summary)
